# Tutorial: Building your Research

The EDSL package allows you to conduct AI-powered research by creating surveys that are answered by large language models.  

This notebook introduces you to the core components of EDSL.
- `Question`: a question you would like to ask, that can be created in a variety of formats
- `Scenario`: optional parameters for your questions
- `Survey`: a collection of questions 
- `Agent`: a persona for an AI agent that will be prompted to answer your questions
- `Model`: the LLM that you would like to use to power your agents
 
 These components can be combined to create empirical research workflows. These workflows include but are not limited to survey research: if you can phrase your task as asking an AI to answer questions, then EDSL will help you with that.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/tutorial_building_your_research.ipynb)

In [1]:
# ! pip install edsl

## Questions

Questions are the core building blocks of EDSL workflows. EDSL provides various question types enable you to get information from LLMs in specific formats, such as textual responses, lists or numerical answers. 

- Each Question type has a construction method and an answer validation method.
- To use a Question, simply import it and create an instance for question you would like to ask.
- Each Question type has an `.example()` method that returns an example of a question type:

In [2]:
from edsl.questions import QuestionFreeText

# Using the .example() method  
QuestionFreeText.example()

QuestionFreeText(question_name = 'how_are_you', question_text = 'How are you?', allow_nonresponse = True)

In [3]:
from edsl.questions import QuestionMultipleChoice

# Creating a question of our own
q1 = QuestionMultipleChoice(
    question_name = "feel_today",
    question_text = "How do you feel today?",
    question_options = ["good", "bad", "neutral"]
)
q1

QuestionMultipleChoice(question_name = 'feel_today', question_text = 'How do you feel today?', question_options = ['good', 'bad', 'neutral'], short_names_dict = {})

## Surveys
Surveys are collections of questions. Surveys are collections of questions. By default, each question is asked serially, and respondents have no memory of previous questions.



In [4]:
from edsl import Survey

# the example survey has three questions
Survey.example()

Survey(questions=[QuestionMultipleChoice(question_name = 'q0', question_text = 'Do you like school?', question_options = ['yes', 'no'], short_names_dict = {}), QuestionMultipleChoice(question_name = 'q1', question_text = 'Why not?', question_options = ['killer bees in cafeteria', 'other'], short_names_dict = {}), QuestionMultipleChoice(question_name = 'q2', question_text = 'Why?', question_options = ['**lack*** of killer bees in cafeteria', 'other'], short_names_dict = {})], name=None)

In [5]:
# create a survey with two questions
q2 = QuestionFreeText(
    question_name = "favorite_fruit",
    question_text = "What is your favorite fruit?"
)

survey = Survey(questions = [q1, q2])

survey

Survey(questions=[QuestionMultipleChoice(question_name = 'feel_today', question_text = 'How do you feel today?', question_options = ['good', 'bad', 'neutral'], short_names_dict = {}), QuestionFreeText(question_name = 'favorite_fruit', question_text = 'What is your favorite fruit?', allow_nonresponse = False)], name=None)

## Agents
Agents consist of personas that we prompt LLMs to reference in responding to questions. An agent is created by passing a persona with any number of traits. We can optionally give an agent a name, or use the default name that is created when a survey is administered to the agent.

Here we show the `.example()` agent and then construct a set of agents with different personas:

In [6]:
from edsl import Agent 

# the example agent has three traits: "age", "hair", and "height"
Agent.example()

Agent(traits = {'age': 22, 'hair': 'brown', 'height': 5.5})

In [7]:
# constructing three agents with a single "profession" trait
professions = ["You are an engineer.", "You are a doctor.", "You are a student."]

agents = [Agent(traits = {"profession": p}) for p in professions]
agents

[Agent(traits = {'profession': 'You are an engineer.'}),
 Agent(traits = {'profession': 'You are a doctor.'}),
 Agent(traits = {'profession': 'You are a student.'})]

## Models
Models allow you to specify the LLMs that you want to use for your survey. We can see the available models and select one or more of them when administering a survey:

In [8]:
from edsl import Model

# see the available models
available_models = Model.available()
print(f"Available models: {available_models}")

# select one model, and see its parameters
model = Model('gpt-3.5-turbo')
model

Available models: ['claude-3-haiku-20240307', 'claude-3-opus-20240229', 'claude-3-sonnet-20240229', 'dbrx-instruct', 'gemini_pro', 'gpt-3.5-turbo', 'gpt-4-1106-preview', 'llama-2-13b-chat-hf', 'llama-2-70b-chat-hf', 'mixtral-8x7B-instruct-v0.1']


LanguageModelOpenAIThreeFiveTurbo(model = 'gpt-3.5-turbo', parameters={'temperature': 0.5, 'max_tokens': 1000, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'logprobs': False, 'top_logprobs': 3})

## Scenarios

Scenarios give you an easy way to parameterize your questions. 

To use scenarios:
- Parameterize your question by using the ```{{ parameter_name }} ``` notation in the question text
- Create a scenario by passing a dictionary of parameters to the ```Scenario``` constructor

In [9]:
from edsl import Scenario

# Parameterize a question
q2 = QuestionFreeText(
    question_name = "favorite_thing",
    question_text = "What is your favorite {{ thing }}?"
)

things = ["color", "fruit", "day of the week"]
scenarios = [Scenario({"thing": t}) for t in things]

## Running Surveys and getting Results

To run a survey, you simply need to call its `.run()` method. To add agents, models, and scenarios to your survey, you can use the `.by()` method. By default, the survey is administered to Agents without any traits, and using OpenAI's GPT-4 model.

Results are the output of administering your survey. The Results object does not only contain the answers to your questions, but it also comes with methods that help you to explore and visualize your data.

In [10]:
# create a survey with the components we constructed throughout, and run it
survey = Survey([q1,q2])
results = survey.by(scenarios).by(agents).by(model).run()

results.select("profession","feel_today", "favorite_thing").print()

agent.profession,answer.feel_today,answer.favorite_thing
You are a doctor.,good,My favorite fruit is mango.
You are an engineer.,good,My favorite color is blue.
You are a student.,good,My favorite fruit is mango.
You are an engineer.,good,My favorite fruit is definitely mango. I love its sweet and juicy flavor.
You are an engineer.,good,My favorite day of the week is Friday because it marks the end of the workweek and the start of the weekend.
You are a student.,good,My favorite day of the week is Friday because it marks the end of the work week and the start of the weekend.
You are a doctor.,good,My favorite day of the week is Saturday because I get to relax and spend time with my family and friends.
You are a student.,good,My favorite color is blue.
You are a doctor.,good,My favorite color is blue.


---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>